In [6]:
import xml.etree.ElementTree as ET

In [39]:
tree = ET.parse('final.xml')
root = tree.getroot()

In [8]:
for child in root:
    print(child.tag, child.attrib)

release {'id': '1'}
release {'id': '2'}
release {'id': '3'}


In [16]:

import pandas as pd

import sys
if sys.version_info[0] == 3:
    from urllib.request import urlopen
else:
    from urllib import urlopen


In [ ]:
pip install python3-discogs-client

In [10]:
rows = []

for node in root:
    n_title = node.attrib["id"]
    #n_genre = node.find("genre").text
    rows.append({"title":n_title})
    

In [74]:
import discogs_client

d = discogs_client.Client('ExampleApplication/0.1', user_token = 'WkcTWMWNFmwUFYHrtQIreHRnLmyFZTNnkflNHsgc')


In [77]:
master = []
masters = []

for release in root.iter('master'):
    id = release.attrib.get('id')
    gens = []
    for gen in release.iter('genre'):
        gens.append(gen.text)
    stys = []
    for sty in release.iter('style'):
        stys.append(sty.text)
    uri = d.master(id).images[0]['uri']
    
    master = [id, gens, stys, uri]
    
    masters.append(master)

df = pd.DataFrame(masters, columns=['id', 'genre', 'styles', 'uri'])

print(df)

    id         genre                 styles  \
0  121  [Electronic]  [Hardcore, Breakbeat]   

                                                 uri  
0  https://i.discogs.com/V7w_rfIEDOXNGCclcLctqnrS...  


In [41]:
DISCOGS_KEY = 'OKxXReJmUJvMJRRsaOqr'
DISCOGS_SECRET = 'RormADbxpiujDCXDiBDwFGUFxKrVjkTK'

In [73]:
import discogs_client

d = discogs_client.Client('ExampleApplication/0.1', user_token = 'WkcTWMWNFmwUFYHrtQIreHRnLmyFZTNnkflNHsgc')

num = 121

asd = d.master(num).images[0]

type(asd)
#print(result)

print(asd['uri'])


https://i.discogs.com/V7w_rfIEDOXNGCclcLctqnrS3W1NfA45cGYKGWNlaxg/rs:fit/g:sm/q:90/h:600/w:600/czM6Ly9kaXNjb2dz/LWltYWdlcy9SLTEw/NDAzOC0xMjI3NDQ1/MTc5LmpwZWc.jpeg


In [ ]:
from bs4 import BeautifulSoup
import re
import urllib.request
from urllib.request import urlopen
import pandas as pd

url = open(r"C:\Users\cid2r\Documents\!VSCode\0220dbtest\rel.xml")

soup = BeautifulSoup(url, "html.parser")


data = {'id' : [], 'genre' : []}


for release in soup.select('release'):
    data['id'].append(release['id'])
    for gen in release.select('genre'):
        data['genre'].append(gen)

asdf = pd.DataFrame(data)

print(data)

#print(asdf)


In [19]:


for child in root:
    if(child.find("master_id") != None):
        print("=" * 10)
        print(child.attrib["id"])
        for gens in child.iter("genres"):
            for gen in gens.findall("genre"):
                print(gen.text)
        for mr in child.iter("master_id"):
            print(mr.text)
        ##child.write("output.xml", encoding="utf-8", xml_declaration=True)
        

1
jazz
soul
100
3
rock
300
